In [9]:
import pandas as pd
import re
import requests
import certifi
from bs4 import BeautifulSoup

In [238]:
import pandas as pd # 0.23.0
import requests     # 2.19.1
import io

url = 'http://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?csv.x=yes'

payload = {
    'Datefrom'   : '01/Jan/2000',
    #'Dateto'     : '01/Oct/2018',
    'SeriesCodes': 'IUDBEDR',
    'CSVF'       : 'TN',
    'UsingCodes' : 'Y',
    'VPD'        : 'Y',
    'VFD'        : 'N'
}
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/54.0.2840.90 '
                  'Safari/537.36'
}
response = requests.get(url, params=payload, headers=headers)
tas = pd.read_csv(io.BytesIO(response.content),names=['Fecha','Tasa'],skiprows=1)
tas['Fecha']=pd.to_datetime(tas['Fecha'], format='%d %b %Y')#.dt.strftime('%d-%m-%Y')
tas.set_index('Fecha',inplace=True)
#tas=tas.resample('M').last()
tas_t=tas['Tasa'].iloc[-1]
tas_t1=tas['Tasa'].iloc[-2]
#Metrica

url='https://www.ons.gov.uk/generator?format=csv&uri=/economy/inflationandpriceindices/timeseries/l55o/mm23'
response=requests.get(url)
data = io.StringIO(response.text)
inf = pd.read_csv(data,skiprows=316,names=['Fecha','Inflacion'])
inf.columns=['Fecha','Inflacion']
inf['Fecha']=pd.to_datetime(inf['Fecha'], format='%Y %b')#.dt.strftime('%d-%m-%Y')
#metrica

url='https://www.ons.gov.uk/generator?format=csv&uri=/employmentandlabourmarket/peoplenotinwork/unemployment/timeseries/mgsx/lms'
response=requests.get(url)
data = io.StringIO(response.text)
une = pd.read_csv(data,skiprows=621,names=['Fecha','Inflacion'])
une.columns=['Fecha','Desempleo']
une['Fecha']=pd.to_datetime(une['Fecha'], format='%Y %b')#.dt.strftime('%d-%m-%Y')

In [240]:
#tas.index=pd.to_datetime(tas.index)
tas.index[-1].strftime('%b')

'May'

In [224]:
tas

,Fecha,Tasa
0,04-01-2000,5.50
1,05-01-2000,5.50
2,06-01-2000,5.50
3,07-01-2000,5.50
4,10-01-2000,5.50
...,...,...
6160,22-05-2024,5.25
6161,23-05-2024,5.25
6162,24-05-2024,5.25
6163,28-05-2024,5.25


In [216]:
_.head(12)

,Fecha,Tasa
0,04 Jan 2000,5.50
1,05 Jan 2000,5.50
2,06 Jan 2000,5.50
3,07 Jan 2000,5.50
4,10 Jan 2000,5.50
5,11 Jan 2000,5.50
6,12 Jan 2000,5.50
7,13 Jan 2000,5.75
8,14 Jan 2000,5.75
9,17 Jan 2000,5.75


In [178]:
from io import StringIO
ulr='https://www.ons.gov.uk/generator?format=csv&uri=/economy/inflationandpriceindices/timeseries/l55o/mm23'
response=requests.get(ulr)
data = StringIO(response.text)
inf = pd.read_csv(data,skiprows=316,names=['Fecha','Inflacion'])
inf.columns=['Fecha','Inflacion']
inf['Fecha']=pd.to_datetime(inf['Fecha'], format='%Y %b')
inf

,Fecha,Inflacion
0,2000-01-01,1.2
1,2000-02-01,1.2
2,2000-03-01,1.0
3,2000-04-01,0.9
4,2000-05-01,1.0
...,...,...
287,2023-12-01,4.2
288,2024-01-01,4.2
289,2024-02-01,3.8
290,2024-03-01,3.8


In [187]:
url='https://www.ons.gov.uk/generator?format=csv&uri=/employmentandlabourmarket/peoplenotinwork/unemployment/timeseries/mgsx/lms'
response=requests.get(url)
data = StringIO(response.text)
une = pd.read_csv(data,skiprows=621,names=['Fecha','Inflacion'])
une.columns=['Fecha','Desempleo']
une['Fecha']=pd.to_datetime(une['Fecha'], format='%Y %b')
une

,Fecha,Desempleo
0,2000-01-01,5.8
1,2000-02-01,5.8
2,2000-03-01,5.7
3,2000-04-01,5.6
4,2000-05-01,5.5
...,...,...
285,2023-10-01,3.9
286,2023-11-01,3.8
287,2023-12-01,4.0
288,2024-01-01,4.2


In [5]:
import requests
import json
import pandas as pd
# URL del endpoint de la API
url = "https://www.presupuestoabierto.gob.ar/api/v1/credito?format=csv"

# Cabeceras de la solicitud
headers = {
    'Authorization': 'ee7f6d62-90ad-4a31-8db0-844acf12ee27',  # Reemplaza con tu token
    'Content-Type': 'application/json'
}

# Datos de la solicitud
data = {
    "title": "Credito vigente por jurisdiccion",
    "columns": [
        "impacto_presupuestario_mes",
        "jurisdiccion_desc",
        "finalidad_desc",
        "fuente_financiamiento_desc",
        "ubicacion_geografica_desc",
        "credito_vigente"
    ]
}

# Realizar la solicitud POST a la API
response = requests.post(url, headers=headers, data=json.dumps(data))


with open('presupuestos.csv', 'w', encoding='utf-8') as file:
    file.write(response.text)
df=pd.read_csv('presupuestos.csv',encoding='utf-8')
df

ConnectTimeout: HTTPSConnectionPool(host='www.presupuestoabierto.gob.ar', port=443): Max retries exceeded with url: /api/v1/credito?format=csv (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002A336871390>, 'Connection to www.presupuestoabierto.gob.ar timed out. (connect timeout=None)'))

In [8]:
import pandas as pd
import json
provincias={'Ciudad Autónoma de Buenos Aires':'Capital Federal',
 'Provincia de Buenos Aires':'Buenos Aires',
 'Provincia de Catamarca':'Catamarca',
 'Provincia de Corrientes':'Corrientes',
 'Provincia de Córdoba':'Córdoba',
 'Provincia de Entre Ríos':'Entre Ríos',
 'Provincia de Formosa':'Formosa',
 'Provincia de Jujuy':'Jujuy',
 'Provincia de La Pampa':'La Pampa',
 'Provincia de La Rioja':'La Rioja',
 'Provincia de Mendoza':'Mendoza',
 'Provincia de Misiones':'Misiones',
 'Provincia de Río Negro':'Río Negro',
 'Provincia de Salta':'Salta',
 'Provincia de San Juan':'San Juan',
 'Provincia de San Luis':'San Luis',
 'Provincia de Santa Cruz':'Santa Cruz',
 'Provincia de Santa Fe':'Santa Fe',
 'Provincia de Santiago del Estero':'Santiago del Estero',
 'Provincia de Tierra del Fuego, Antártida e Islas del Atlántico Sur':'Tierra del Fuego, Antártida e Islas del Atlántico Sur',
 'Provincia de Tucumán':'Tucumán',
 'Provincia del Chaco':'Chaco',
 'Provincia del Chubut':'Chubut',
 'Provincia del Neuquén':'Neuquén'}
df=pd.read_csv('presupuestos.csv',encoding='utf-8')

df1=df.groupby(by=['ubicacion_geografica_desc']).sum()#.reset_index()
df1['%']=100*df1['credito_vigente']/df1['credito_vigente'].sum()
extras=df1.iloc[0:4]
df1=df1.drop(index=['Binacional','No Clasificado','Interprovincial','Nacional'],)
df1=df1.reset_index()
df1['ubicacion_geografica_desc'] = df1['ubicacion_geografica_desc'].replace(provincias)


geo=json.load(open('provincias.geojson', encoding='utf-8'))
provincias_geo_df = pd.json_normalize(geo['features'])
#provincias_geo_df['properties.nombre'] = 'Provincia de ' + provincias_geo_df['properties.nombre']

merged_df = provincias_geo_df.merge(df1, left_on='properties.nombre', right_on='ubicacion_geografica_desc')
merged_df

C:\Users\feder\AppData\Local\Temp\ipykernel_7100\855307051.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df1=df.groupby(by=['ubicacion_geografica_desc']).sum()#.reset_index()


,type,geometry.type,geometry.coordinates,properties.id,properties.nombre,properties.nombre_completo,properties.fuente,properties.categoria,properties.centroide.lon,properties.centroide.lat,properties.iso_id,properties.iso_nombre,ubicacion_geografica_desc,impacto_presupuestario_mes,credito_vigente,%
0,Feature,Polygon,"[[[-70.393406682, -36.156241237], [-70.3949855...",58,Neuquén,Provincia del Neuquén,IGN,Provincia,-70.119897,-38.641983,AR-Q,Neuquén,Neuquén,833,4.187527e+05,0.759584
1,Feature,Polygon,"[[[-67.055470113, -31.856052635], [-67.0059302...",74,San Luis,Provincia de San Luis,IGN,Provincia,-66.025231,-33.761104,AR-D,San Luis,San Luis,720,4.359788e+05,0.790831
2,Feature,Polygon,"[[[-61.046389631, -27.9980009399999], [-58.884...",82,Santa Fe,Provincia de Santa Fe,IGN,Provincia,-60.950687,-30.708823,AR-S,Santa Fe,Santa Fe,967,1.664028e+06,3.018416
3,Feature,Polygon,"[[[-68.521356179, -27.8179169699999], [-68.510...",46,La Rioja,Provincia de La Rioja,IGN,Provincia,-67.181758,-29.684937,AR-F,La Rioja,La Rioja,701,3.883016e+05,0.704349
4,Feature,Polygon,"[[[-68.5053654399999, -25.168509816], [-67.864...",10,Catamarca,Provincia de Catamarca,IGN,Provincia,-66.947897,-27.335954,AR-K,Catamarca,Catamarca,686,4.199475e+05,0.761752
5,Feature,Polygon,"[[[-65.3591542509999, -26.0594376889999], [-65...",90,Tucumán,Provincia de Tucumán,IGN,Provincia,-65.364766,-26.948284,AR-T,Tucumán,Tucumán,855,1.022367e+06,1.854494
6,Feature,Polygon,"[[[-62.3243395499999, -24.1231840889999], [-62...",22,Chaco,Provincia del Chaco,IGN,Provincia,-60.765116,-26.386987,AR-H,Chaco,Chaco,820,6.173304e+05,1.119789
7,Feature,Polygon,"[[[-62.312005272, -22.4866595899999], [-62.290...",34,Formosa,Provincia de Formosa,IGN,Provincia,-59.932190,-24.895087,AR-P,Formosa,Formosa,685,3.014772e+05,0.546856
8,Feature,MultiPolygon,"[[[[-67.593607919, -46.017442156], [-67.585965...",78,Santa Cruz,Provincia de Santa Cruz,IGN,Provincia,-69.955762,-48.815547,AR-Z,Santa Cruz,Santa Cruz,843,4.151853e+05,0.753113
9,Feature,MultiPolygon,"[[[[-67.75056885, -41.99998806], [-65.06925398...",26,Chubut,Provincia del Chubut,IGN,Provincia,-68.526736,-43.788627,AR-U,Chubut,Chubut,862,4.780174e+05,0.867086


In [171]:
provincias={'Ciudad Autónoma de Buenos Aires':'Capital Federal',
 'Provincia de Buenos Aires':'Buenos Aires',
 'Provincia de Catamarca':'Catamarca',
 'Provincia de Corrientes':'Corrientes',
 'Provincia de Córdoba':'Córdoba',
 'Provincia de Entre Ríos':'Entre Ríos',
 'Provincia de Formosa':'Formosa',
 'Provincia de Jujuy':'Jujuy',
 'Provincia de La Pampa':'La Pampa',
 'Provincia de La Rioja':'La Rioja',
 'Provincia de Mendoza':'Mendoza',
 'Provincia de Misiones':'Misiones',
 'Provincia de Río Negro':'Río Negro',
 'Provincia de Salta':'Salta',
 'Provincia de San Juan':'San Juan',
 'Provincia de San Luis':'San Luis',
 'Provincia de Santa Cruz':'Santa Cruz',
 'Provincia de Santa Fe':'Santa Fe',
 'Provincia de Santiago del Estero':'Santiago del Estero',
 'Provincia de Tierra del Fuego, Antártida e Islas del Atlántico Sur':'Tierra del Fuego, Antártida e Islas del Atlántico Sur',
 'Provincia de Tucumán':'Tucumán',
 'Provincia del Chaco':'Chaco',
 'Provincia del Chubut':'Chubut',
 'Provincia del Neuquén':'Neuquén'}
df=pd.read_csv('presupuestos.csv',encoding='utf-8')

df1=df.groupby(by=['ubicacion_geografica_desc']).sum()#.reset_index()
df1['%']=100*df1['credito_vigente']/df1['credito_vigente'].sum()
extras=df1.iloc[0:4]
df1=df1.drop(index=['Binacional','No Clasificado','Interprovincial','Nacional'],)
df1=df1.reset_index()
df1['ubicacion_geografica_desc'] = df1['ubicacion_geografica_desc'].replace(provincias)


geo=json.load(open('ProvinciasArgentina.geojson', encoding='utf-8'))
provincias_geo_df = pd.json_normalize(geo['features'])
#provincias_geo_df['properties.nombre'] = 'Provincia de ' + provincias_geo_df['properties.nombre']

merged_df = provincias_geo_df.merge(df1, left_on='properties.nombre', right_on='ubicacion_geografica_desc')

fig = px.choropleth_mapbox(
    merged_df,
    geojson=geo,
    locations='ubicacion_geografica_desc',
    featureidkey='properties.nombre',
    color='%', # Asegúrate de que esta columna contiene el gasto fiscal
    hover_name='properties.nombre',
    custom_data=['properties.nombre','credito_vigente','%'],
    color_continuous_scale='magma',
                           mapbox_style= "carto-positron" , # formatos de diseño del mapa : "carto-positron", "carto-positron",   "white-bg",
                           zoom=2.6, center = {"lat": -38.40, "lon": -63.60},
                           opacity=1,
                           labels={'promedio acessos por cada 100 hogares':'acceso a internet'},
                           color_discrete_sequence=["blue"],
                          )
fig.update_traces(
    marker_line_width=1.5,
    marker_line_color='black',
    hovertemplate="<br>".join([
        "<b>%{customdata[0]}</b>",  # Asegúrate de cerrar la etiqueta <b> correctamente aquí
        "Presupuesto Brindado: %{customdata[1]}",
        "Proporción en relación al presupuesto total: %{customdata[2]:.2f}%"
    ])
)
# Actualizar el layout del mapa
#fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    mapbox=dict(
        center={"lat": -38.4161, "lon": -63.6167},
        style="white-bg",
        zoom=2.65,
        layers=[
            dict(
                below='traces',
                type='fill',
                source=geo,
                color="lightblue"
            )
        ]
    ),
    showlegend=False,
    margin=dict(t=0, b=0, l=0, r=0),
    geo=dict(
        showland=False,  # No mostrar etiquetas de países
        showcountries=False,  # No mostrar bordes de países
    )
)

fig.show()

C:\Users\feder\AppData\Local\Temp\ipykernel_14436\3697934728.py:27: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [172]:
import json
import folium

# Leer el archivo JSON
with open('provincias.json', 'r') as file:
    data = json.load(file)

# Crear el mapa centrado en Argentina
m = folium.Map(location=[-38.4161, -63.6167], zoom_start=4)

# Agregar las provincias al mapa
for feature in data['features']:
    nombre = feature['properties']['nombre']
    coords = feature['geometry']['coordinates']
    folium.Polygon(
        locations=[(lat, lon) for lon, lat in coords[0]],
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        popup=nombre
    ).add_to(m)

# Guardar el mapa en un archivo HTML
map_path = "mapa_argentina.html"
m.save(map_path)

map_path


'mapa_argentina.html'

In [83]:
url = "https://www.presupuestoabierto.gob.ar/api/v1/pef?format=csv"

# Cabeceras de la solicitud
headers = {
    'Authorization': 'ee7f6d62-90ad-4a31-8db0-844acf12ee27',  # Reemplaza con tu token
    'Content-Type': 'application/json'
}

# Datos de la solicitud
data = {
    "title": "Programación y ejecución física del primer trimestre",
    
    "columns": [
"ejercicio_presupuestario",
"trimestre",
"finalidad_id",
"finalidad_desc",
"funcion_id",
"funcion_desc",
"sector_id",
"sector_desc",
"subsector_id",
"subsector_desc",
"caracter_id",
"caracter_desc",
"jurisdiccion_id",
"jurisdiccion_desc",
"subjurisdiccion_id",
"subjurisdiccion_desc",
"entidad_id",
"entidad_desc",
"servicio_id",
"servicio_desc",
"programa_id",
"programa_desc",
"subprograma_id",
"subprograma_desc",
"proyecto_id",
"proyecto_desc",
"actividad_id",
"actividad_desc",
"obra_id",
"obra_desc",
"medicion_fisica_id",
"medicion_fisica_desc",
"tipo_medicion_fisica",
"unidad_medida_id",
"unidad_medida_desc",
"ubicacion_geografica_id",
"ubicacion_geografica_desc",
"totalizador_avance_fisico",
"programacion_inicial_DA",
"programacion_inicial_ajustada",
"programacion_anual_vig_cierre",
"programacion_anual_vig_trim",
"programacion_trim",
"programacion_acumulada_trim",
"ejecutado_vigente_trim",
"ejecutado_acumulado_trim",
"ejecutado_cierre_acum_trim",
"ejecucion_anual_de_cierre",
"tipo_causa_desvio",
"causa_desvio",
"causa_desvio_detalle",
"causa_desvio_comentario",
"porc_desvio_acum_trim",
"ultima_actualizacion_fecha"
],
    "ejercicios":[2024]
}


# Realizar la solicitud POST a la API
response = requests.post(url, headers=headers, data=json.dumps(data))


In [84]:
response

<Response [200]>

In [85]:
if response.status_code == 200:
    # Guardar la respuesta directamente en un archivo CSV
    with open('credito_vigente_por_jurisdiccion.csv', 'w', encoding='utf-8') as file:
        file.write(response.text)
    print("Datos guardados en credito_vigente_por_jurisdiccion.csv")
else:
    print(f"Error {response.status_code}: {response.text}")

Datos guardados en credito_vigente_por_jurisdiccion.csv


In [89]:
df=pd.read_csv('credito_vigente_por_jurisdiccion.csv')
df.columns#.iloc[1]

Index(['ejercicio_presupuestario', 'trimestre', 'finalidad_id',
       'finalidad_desc', 'funcion_id', 'funcion_desc', 'sector_id',
       'sector_desc', 'subsector_id', 'subsector_desc', 'caracter_id',
       'caracter_desc', 'jurisdiccion_id', 'jurisdiccion_desc',
       'subjurisdiccion_id', 'subjurisdiccion_desc', 'entidad_id',
       'entidad_desc', 'servicio_id', 'servicio_desc', 'programa_id',
       'programa_desc', 'subprograma_id', 'subprograma_desc', 'proyecto_id',
       'proyecto_desc', 'actividad_id', 'actividad_desc', 'obra_id',
       'obra_desc', 'medicion_fisica_id', 'medicion_fisica_desc',
       'tipo_medicion_fisica', 'unidad_medida_id', 'unidad_medida_desc',
       'ubicacion_geografica_id', 'ubicacion_geografica_desc',
       'totalizador_avance_fisico', 'programacion_inicial_DA',
       'programacion_inicial_ajustada', 'programacion_anual_vig_cierre',
       'programacion_anual_vig_trim', 'programacion_trim',
       'programacion_acumulada_trim', 'ejecutado_vige

In [91]:
df['subjurisdiccion_desc'].unique()

array(['Poder Legislativo Nacional', 'Poder Judicial de la Nación',
       'Ministerio de Justicia', 'Procuración del Tesoro de la Nación',
       'Secretaría Legal y Técnica', 'Secretaría de Prensa',
       'Jefatura de Gabinete de Ministros', 'Ministerio de Economía',
       'Ministerio de Infraestructura',
       'Ministerio de Relaciones Exteriores, Comercio Internacional y Culto',
       'Ministerio del Interior', 'Presidencia de la Nación',
       'Ministerio de Defensa',
       'Estado Mayor General del Ejército Argentino',
       'Estado Mayor General de la Armada Argentina',
       'Estado Mayor General de la Fuerza Aérea Argentina',
       'Estado Mayor Conjunto de las Fuerzas Armadas (EMCO)',
       'Ministerio de Seguridad', 'Policía Federal Argentina',
       'Gendarmería Nacional', 'Prefectura Naval Argentina',
       'Policía de Seguridad Aeroportuaria',
       'Servicio Penitenciario Federal', 'Ministerio de Salud',
       'Ministerio de Capital Humano',
       'Secreta

In [54]:
url = "https://www.presupuestoabierto.gob.ar/api/v1/transversal_financiero?format=csv"

# Cabeceras de la solicitud
headers = {
    'Authorization': 'ee7f6d62-90ad-4a31-8db0-844acf12ee27',  # Reemplaza con tu token
    'Content-Type': 'application/json'
}

# Datos de la solicitud
data = {
    "title": "Programación y ejecución física del primer trimestre",
    
    "columns": [
"ejercicio_presupuestario",
"sector_id",
"sector_desc",
"subsector_id",
"subsector_desc",
"caracter_id",
"caracter_desc",
"jurisdiccion_id",
"jurisdiccion_desc",
"subjurisdiccion_id",
"subjurisdiccion_desc",
"entidad_id",
"entidad_desc",
"servicio_id",
"servicio_desc",
"programa_id",
"programa_desc",
"subprograma_id",
"subprograma_desc",
"proyecto_id",
"proyecto_desc",
"actividad_id",
"actividad_desc",
"obra_id",
"obra_desc",
"finalidad_id",
"finalidad_desc",
"funcion_id",
"funcion_desc",
"fuente_financiamiento_id",
"fuente_financiamiento_desc",
"ubicacion_geografica_id",
"ubicacion_geografica_desc",
"inciso_id",
"inciso_desc",
"principal_id",
"principal_desc",
"parcial_id",
"parcial_desc",
"subparcial_id",
"subparcial_desc",
"etiqueta_id",
"etiqueta_desc",
"ponderacion_credito",
"codigo_bapin_id",
"codigo_bapin_desc",
"credito_inicial",
"credito_inicial_ponderado",
"credito_vigente",
"credito_vigente_ponderado",
"credito_ejecutado",
"credito_ejecutado_ponderado"
],
    
    "ejercicios":[2024]
}


# Realizar la solicitud POST a la API
response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
    # Guardar la respuesta directamente en un archivo CSV
    with open('transversal_financiero.csv', 'w', encoding='utf-8') as file:
        file.write(response.text)
    df=pd.read_csv('transversal_financiero.csv')
df.iloc[1]

In [78]:
df['subjurisdiccion_desc'].unique()

array(['Poder Legislativo Nacional', 'Poder Judicial de la Nación',
       'Ministerio de Justicia', 'Procuración del Tesoro de la Nación',
       'Secretaría Legal y Técnica', 'Secretaría de Prensa',
       'Jefatura de Gabinete de Ministros', 'Ministerio de Economía',
       'Ministerio de Infraestructura',
       'Ministerio de Relaciones Exteriores, Comercio Internacional y Culto',
       'Ministerio del Interior', 'Presidencia de la Nación',
       'Ministerio de Defensa',
       'Estado Mayor General del Ejército Argentino',
       'Estado Mayor General de la Armada Argentina',
       'Estado Mayor General de la Fuerza Aérea Argentina',
       'Estado Mayor Conjunto de las Fuerzas Armadas (EMCO)',
       'Ministerio de Seguridad', 'Policía Federal Argentina',
       'Gendarmería Nacional', 'Prefectura Naval Argentina',
       'Policía de Seguridad Aeroportuaria',
       'Servicio Penitenciario Federal', 'Ministerio de Salud',
       'Ministerio de Capital Humano',
       'Secreta

In [79]:
url = "https://www.presupuestoabierto.gob.ar/api/v1/credito?format=csv"

# Cabeceras de la solicitud
headers = {
    'Authorization': 'ee7f6d62-90ad-4a31-8db0-844acf12ee27',  # Reemplaza con tu token
    'Content-Type': 'application/json'
}

# Datos de la solicitud
data = {
    "title": "Programación y ejecución física del primer trimestre",
    
    "columns": [
"impacto_presupuestario_fecha",
"impacto_presupuestario_anio",
"impacto_presupuestario_mes",
"ejercicio_presupuestario",
"sector_id",
"sector_desc",
"subsector_id",
"subsector_desc",
"caracter_id",
"caracter_desc",
"jurisdiccion_id",
"jurisdiccion_desc",
"subjurisdiccion_id",
"subjurisdiccion_desc",
"entidad_id",
"entidad_desc",
"servicio_id",
"servicio_desc",
"programa_id",
"programa_desc",
"subprograma_id",
"subprograma_desc",
"proyecto_id",
"proyecto_desc",
"actividad_id",
"actividad_desc",
"obra_id",
"obra_desc",
"finalidad_id",
"finalidad_desc",
"funcion_id",
"funcion_desc",
"inciso_id",
"inciso_desc",
"principal_id",
"principal_desc",
"parcial_id",
"parcial_desc",
"subparcial_id",
"subparcial_desc",
"clasificador_economico_8_digitos_id",
"clasificador_economico_8_digitos_desc",
"fuente_financiamiento_id",
"fuente_financiamiento_desc",
"ubicacion_geografica_id",
"ubicacion_geografica_desc",
"unidad_ejecutora_id",
"unidad_ejecutora_desc",
"prestamo_externo_id",
"prestamo_externo_desc",
"codigo_bapin_id",
"codigo_bapin_desc",
"credito_presupuestado",
"credito_vigente",
"credito_comprometido",
"credito_devengado",
"credito_pagado",
"ultima_actualizacion_fecha"
],
    
    "ejercicios":[2023]
}


# Realizar la solicitud POST a la API
response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
    # Guardar la respuesta directamente en un archivo CSV
    with open('credito.csv', 'w', encoding='utf-8') as file:
        file.write(response.text)
    df=pd.read_csv('credito.csv')
df.iloc[1]

ejercicio_presupuestario                                                      2024
trimestre                                                                        1
finalidad_id                                                                     1
finalidad_desc                                        ADMINISTRACION GUBERNAMENTAL
funcion_id                                                                       1
funcion_desc                                                           Legislativa
sector_id                                                                        1
sector_desc                                  Sector Público Nacional no Financiero
subsector_id                                                                     1
subsector_desc                                             Administración Nacional
caracter_id                                                                      1
caracter_desc                                               Administración Central
juri

In [ ]:
url = "https://www.presupuestoabierto.gob.ar/api/v1/recurso?format=csv"

# Cabeceras de la solicitud
headers = {
    'Authorization': 'ee7f6d62-90ad-4a31-8db0-844acf12ee27',  # Reemplaza con tu token
    'Content-Type': 'application/json'
}

# Datos de la solicitud
data = {
    "title": "Programación y ejecución física del primer trimestre",
    
    "columns": [
"impacto_presupuestario_fecha",
"impacto_presupuestario_anio",
"impacto_presupuestario_mes",
"ejercicio_presupuestario",
"sector_id",
"sector_desc",
"subsector_id",
"subsector_desc",
"caracter_id",
"caracter_desc",
"jurisdiccion_id",
"jurisdiccion_desc",
"subjurisdiccion_id",
"subjurisdiccion_desc",
"entidad_id",
"entidad_desc",
"servicio_id",
"servicio_desc",
"tipo_id",
"tipo_desc",
"clase_id",
"clase_desc",
"concepto_id",
"concepto_desc",
"subconcepto_id",
"subconcepto_desc",
"fuente_financiamiento_id",
"fuente_financiamiento_desc",
"clasificador_economico_8_digitos_id",
"clasificador_economico_8_digitos_desc",
"prestamo_externo_id",
"prestamo_externo_desc",
"recurso_inicial",
"recurso_vigente",
"recurso_ingresado_percibido",
"ultima_actualizacion_fecha"
],
    
    "ejercicios":[2023]
}


# Realizar la solicitud POST a la API
response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
    # Guardar la respuesta directamente en un archivo CSV
    with open('recurso.csv', 'w', encoding='utf-8') as file:
        file.write(response.text)
    df=pd.read_csv('recurso.csv')
df.iloc[1]

,Titulos Publicos,Titulos Publicos-Moneda Nacional,Deuda no ajustable por CER,Deuda ajustable por CER,Titulos Publicos-Moneda Extranjera,Letras,Letras-Moneda Nacional,Letras-Moneda Extranjera,Prestamos,Total Deuda Bruta,Otros
2019-01-31,216771.7,43886.0,23231.6,20654.4,172870.8,35631.3,14952.1,20679.2,61442.8,335661.3,21815.5
2019-02-28,219175.7,44347.6,23465.5,20882.1,174813.2,35102.3,14451.3,20651.0,61420.0,334477.0,18779.0
2019-03-31,213330.2,41501.1,20538.2,20962.9,171814.3,34653.1,14081.6,20571.5,60737.9,324898.3,16177.1
2019-04-30,212709.8,43277.1,21581.7,21695.4,169417.9,33999.5,14259.7,19739.8,71608.7,334322.9,16004.9
2019-05-31,210302.7,43454.1,21305.9,22148.2,166833.8,34358.7,14934.9,19423.8,70448.5,329930.6,14820.7
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31,263424.7,85520.0,43313.8,42206.2,177890.7,10974.9,5097.1,5877.8,82239.7,370673.4,14034.1
2024-01-31,272907.6,93370.4,43288.4,50082.0,179523.1,10022.5,4175.6,5846.9,84237.9,381271.9,14103.9
2024-02-29,280981.2,109034.4,38836.7,70197.7,171932.5,8073.5,2212.1,5861.4,83198.7,386536.8,14283.4
2024-03-31,300792.8,128816.9,16342.5,112474.4,171961.4,8252.5,2391.1,5861.4,82788.8,403043.7,11209.6


In [126]:

# Descargar el archivo
response = requests.get(url)
response

<Response [200]>

Unnamed: 2     335661.3
Unnamed: 3       334477
Unnamed: 4     324898.3
Unnamed: 5     334322.9
Unnamed: 6     329930.6
                 ...   
Unnamed: 61    370673.4
Unnamed: 62    381271.9
Unnamed: 63    386536.8
Unnamed: 64    403043.7
Unnamed: 65    414056.1
Name: A- DEUDA BRUTA ( I + II  + III), Length: 64, dtype: object

Unnamed: 2     61442.8
Unnamed: 3     61420.0
Unnamed: 4     60737.9
Unnamed: 5     71608.7
Unnamed: 6     70448.5
                ...   
Unnamed: 61    82239.7
Unnamed: 62    84237.9
Unnamed: 63    83198.7
Unnamed: 64    82788.8
Unnamed: 65    80669.4
Name: PRÉSTAMOS, Length: 64, dtype: object

In [ ]:
columnas_originales=["A- DEUDA BRUTA ( I + II  + III)",
"nan",
"B- DEUDA BRUTA (EXCLUIDA LA ELEGIBLE PENDIENTE DE REESTRUCTURACIÓN) ( I + II )",
"nan",
"nan",
"I- DEUDA EN SITUACIÓN DE PAGO NORMAL",
"nan",
"        MEDIANO Y LARGO PLAZO",
"nan",
"TÍTULOS PÚBLICOS1",
"nan",
"TP-Moneda nacional",
"Deuda no ajustable por CER",
"BONO CONSOLIDADO 2089",
"BOCON",
"BONAR",
"BONTE",
"BOGATO",
"BOTAPO",
"BONO PGN",
"nan",
"Deuda ajustable por CER",
"BOCON",
"BONCER",
"BOGATO",
"CUASIPAR/$+CER/3,31%/2045",
"DISCOUNT/$+CER/5,83%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/$+CER/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
" - Moneda extranjera ",
"Deuda en dólares estadounidenses",
"LETRA BCRA FDA",
"LETRA BCRA FOI",
"LETRA BCRA",
"BONAR",
"BIRAD",
"BONO R.A. U$S",
"BONO GLOBAL U$S",
"DISCOUNT/U$S/8,28%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/U$S/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en euros",
"BIRAE",
"BONO GLOBAL EUR",
"DISCOUNT/EUR/7,82%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/EUR/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en yenes",
"DISCOUNT/JPY/4,33%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/JPY/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en Franco suizo",
"BIRAF",
"nan",
" - Amparos y excepciones",
"nan",
"LETRAS DEL TESORO",
" - Moneda nacional",
"LETRA CMEA",
"LETES $",
"LECAP",
"nan",
" - Moneda extranjera ",
"LETES - U$S",
"LETRA MENDOZA - U$S",
"LETRAS EN GARANTÍA - U$S",
"nan",
"PRÉSTAMOS",
"nan",
"PRÉSTAMOS GARANTIZADOS",
"nan",
"ORGANISMOS INTERNACIONALES",
"BCIE",
"BIRF",
"BID",
"FONPLATA",
"FIDA",
"CAF",
"OFID",
"FMI",
"BEI",
"nan",
"ORGANISMOS OFICIALES",
"CLUB DE PARIS",
"OTROS BILATERALES",
"nan",
"BANCA COMERCIAL",
"nan",
"PAGARÉS DEL TESORO",
"PAGARÉ 2019 - $",
"PAGARÉ 2021 - CAMMESA - U$S",
"PAGARÉ 2038 - UCP",
"nan",
"AVALES",
"nan",
"ADELANTOS TRANSITORIOS BCRA - Extraordinarios",
"nan",
"        CORTO PLAZO (1)",
"nan",
"ADELANTOS TRANSITORIOS BCRA - Ordinarios",
"nan",
"BANCA COMERCIAL",
"nan",
"PAGARÉS DEL TESORO",
"PAGARE CUT",
"nan",
"AVALES",
"nan",
"TÍTULOS PÚBLICOS2",
"nan",
"LETRAS DEL TESORO",
" - Moneda nacional",
"LETRA FFRH",
"LETRA FFSIT",
"LETRA SRT",
"LETRA ANSES",
"LETES - $",
"LETRA FFDP",
"LETRA FGS",
"LETRA - $",
"LECAP",
"nan",
" - Moneda extranjera ",
"LETRA FGS - U$S"]

In [ ]:
#TÍTULOS PÚBLICOS  =  Titpub1+titpub2
#same for letras del tesoro
nuevas_col={"A- DEUDA BRUTA ( I + II  + III)":'Total Deuda',
"nan",
"B- DEUDA BRUTA (EXCLUIDA LA ELEGIBLE PENDIENTE DE REESTRUCTURACIÓN) ( I + II )",
"nan",
"nan",
"I- DEUDA EN SITUACIÓN DE PAGO NORMAL",
"nan",
"        MEDIANO Y LARGO PLAZO",
"nan",
"TÍTULOS PÚBLICOS",
"nan",
" - Moneda nacional",
"Deuda no ajustable por CER",
"BONO CONSOLIDADO 2089",
"BOCON",
"BONAR",
"BONTE",
"BOGATO",
"BOTAPO",
"BONO PGN",
"nan",
"Deuda ajustable por CER",
"BOCON",
"BONCER",
"BOGATO",
"CUASIPAR/$+CER/3,31%/2045",
"DISCOUNT/$+CER/5,83%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/$+CER/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
" - Moneda extranjera ",
"Deuda en dólares estadounidenses",
"LETRA BCRA FDA",
"LETRA BCRA FOI",
"LETRA BCRA",
"BONAR",
"BIRAD",
"BONO R.A. U$S",
"BONO GLOBAL U$S",
"DISCOUNT/U$S/8,28%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/U$S/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en euros",
"BIRAE",
"BONO GLOBAL EUR",
"DISCOUNT/EUR/7,82%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/EUR/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en yenes",
"DISCOUNT/JPY/4,33%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/JPY/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en Franco suizo",
"BIRAF",
"nan",
" - Amparos y excepciones",
"nan",
"LETRAS DEL TESORO",
" - Moneda nacional",
"LETRA CMEA",
"LETES $",
"LECAP",
"nan",
" - Moneda extranjera ",
"LETES - U$S",
"LETRA MENDOZA - U$S",
"LETRAS EN GARANTÍA - U$S",
"nan",
"PRÉSTAMOS",
"nan",
"PRÉSTAMOS GARANTIZADOS",
"nan",
"ORGANISMOS INTERNACIONALES",
"BCIE",
"BIRF",
"BID",
"FONPLATA",
"FIDA",
"CAF",
"OFID",
"FMI",
"BEI",
"nan",
"ORGANISMOS OFICIALES",
"CLUB DE PARIS",
"OTROS BILATERALES",
"nan",
"BANCA COMERCIAL",
"nan",
"PAGARÉS DEL TESORO",
"PAGARÉ 2019 - $",
"PAGARÉ 2021 - CAMMESA - U$S",
"PAGARÉ 2038 - UCP",
"nan",
"AVALES",
"nan",
"ADELANTOS TRANSITORIOS BCRA - Extraordinarios",
"nan",
"        CORTO PLAZO (1)",
"nan",
"ADELANTOS TRANSITORIOS BCRA - Ordinarios",
"nan",
"BANCA COMERCIAL",
"nan",
"PAGARÉS DEL TESORO",
"PAGARE CUT",
"nan",
"AVALES",
"nan",
"TÍTULOS PÚBLICOS",
"nan",
"LETRAS DEL TESORO",
" - Moneda nacional",
"LETRA FFRH",
"LETRA FFSIT",
"LETRA SRT",
"LETRA ANSES",
"LETES - $",
"LETRA FFDP",
"LETRA FGS",
"LETRA - $",
"LECAP",
"nan",
" - Moneda extranjera ",
"LETRA FGS - U$S"}

In [10]:
import pandas as pd
ica_his=pd.read_csv("His Data/his-ica.csv",delimiter=";")
ica_his['Unnamed: 0'] = pd.to_datetime(ica_his.iloc[:, 0].values, format='%Y-%m-%d')
ica_his.set_index('Unnamed: 0', inplace=True)

ica_his.iloc[156:]

,Expo Totales,PP,MOA,MOI,Combustibles y Energía,Impo Totales,Bienes de capital,Bienes intermedios,Combustibles y Lubricantes,Piezas y acces,Bienes de consumo,Vehículos,Resto
Unnamed: 0,,,,,,,,,,,,,
2005-01-01,2781.105834,508.069267,902.554747,828.721398,541.760422,1901.013368,414.775397,766.878877,86.204834,333.851497,215.259447,72.577251,11.466065
2005-02-01,2606.256268,500.579873,838.735013,831.014551,435.926831,1867.287144,395.336593,739.577823,86.010411,307.208628,216.621515,114.084934,8.447240
2005-03-01,3054.434593,656.222059,972.020431,916.255200,509.936903,2197.568086,469.991165,881.839474,79.084763,401.135034,233.058332,122.527357,9.931961
2005-04-01,3560.857241,939.068339,1039.671591,1008.039167,574.078144,2379.914676,517.107523,875.863612,184.347743,437.072297,243.402660,107.284846,14.835995
2005-05-01,3695.289604,1028.903986,1121.712697,997.304576,547.368345,2475.966974,504.224742,921.800853,192.446590,465.415820,247.432677,133.058200,11.588092
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,5910.000000,1456.000000,2055.000000,1703.000000,697.000000,6885.000000,1166.000000,2671.000000,629.000000,1629.000000,727.000000,18.000000,45.000000
2023-09-01,5751.000000,1426.000000,1810.000000,1934.000000,581.000000,6525.000000,1101.000000,2276.000000,517.000000,1526.000000,682.000000,377.000000,45.000000
2023-10-01,5396.000000,1139.000000,1689.000000,1775.000000,794.000000,5839.000000,971.000000,2075.000000,504.000000,1361.000000,669.000000,216.000000,44.000000


In [11]:
ica_his.iloc[156:].resample('Q').sum()

,Expo Totales,PP,MOA,MOI,Combustibles y Energía,Impo Totales,Bienes de capital,Bienes intermedios,Combustibles y Lubricantes,Piezas y acces,Bienes de consumo,Vehículos,Resto
Unnamed: 0,,,,,,,,,,,,,
2005-03-31,8441.796695,1664.871199,2713.310191,2575.991149,1487.624156,5965.868598,1280.103155,2388.296174,251.300008,1042.195159,664.939294,309.189542,29.845266
2005-06-30,10706.340906,2757.561352,3293.672522,2934.291507,1720.815525,7579.497675,1756.814831,2709.555949,578.143318,1360.712233,745.787358,389.699943,38.784043
2005-09-30,10921.297084,2179.982873,3590.809719,3149.812656,2000.691836,7442.788245,1816.336977,2584.413816,392.238121,1300.820008,858.874044,457.903848,32.201431
2005-12-31,10317.332406,1508.209868,3543.490622,3324.707111,1940.924805,7698.738925,2082.918378,2591.121010,323.709471,1296.442176,929.168149,445.151131,30.228610
2006-03-31,9925.469867,1846.624191,3290.734753,2952.864796,1835.246127,7371.782444,1702.528376,2690.799580,324.163860,1307.607718,829.971676,487.415387,29.295847
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31,21203.000000,5096.000000,7701.000000,6043.000000,2362.000000,16858.000000,2940.000000,6266.000000,1511.000000,3546.000000,2014.000000,492.000000,89.000000
2023-03-31,15899.000000,3622.000000,5518.000000,4712.000000,2048.000000,17243.000000,2597.000000,6672.000000,1874.000000,3641.000000,1938.000000,414.000000,106.000000
2023-06-30,17575.000000,3598.000000,7093.000000,5132.000000,1750.000000,20725.000000,2891.000000,8200.000000,2714.000000,4201.000000,1995.000000,603.000000,124.000000
